In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# A Simple Starter Code for the AJL Competition

@Cindy Deng

---


Hi! This starter code is designed to help you get familiar with basic Kaggle operations and guide you through the basic workflow of a machine learning project. 

The code outlines essential steps including data loading, preprocessing, model building, training, and generating predictions. Each section serves as a foundation, but there are many ways to enhance each step to improve your final model's accuracy. Feel free to experiment with different data augmentation techniques, model architectures, and tuning methods to optimize your final results! Some amazing tutorials are available through your AI Studio course in Canvas / in the 'Resource' section of this Kaggle competition.

Good luck and have fun!

---

## Note - About file path

You could use the cell above to print the names of the file directories and get the following directories:

```
/kaggle/input/bttai-ajl-2025/sample_submission.csv
/kaggle/input/bttai-ajl-2025/train.csv
/kaggle/input/bttai-ajl-2025/test.csv
/kaggle/input/bttai-ajl-2025/test/test/e0374ae6c1362ff183cfba28ded5421b.jpg
/kaggle/input/bttai-ajl-2025/test/test/437159c605260bdd079af230566af291.jpg
...
...
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/11271bdf2598afdd4260db3125e1f6a5.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/732819951dcf2b53d15ea7b8bb123b71.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/6dcc7a8abb5e1c6e670101f4b6231246.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/e63c3b3f0ab8905e204fe467cc7411f9.jpg
...
...
```



## 1. Import Necessary Libraries

In [2]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


# Explanation:
# - pandas and numpy: for data manipulation
# - sklearn: for splitting data and encoding labels
# - tensorflow.keras: for building and training the neural network


## 2. Load Data

Make sure to verify the file paths if you're running on a different platform.

In [3]:
# 2. Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

In [4]:
# Check the first few rows to understand the structure
train_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,NaN,12,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,1 Diagnostic,56,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,NaN,34,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,NaN,12,prurigo-nodularis/c77d6c895f05fea73a8f37043070...


## 3. Data Preprocessing


This section demonstrates basic preprocessing techniques. To enhance data quality and model performance, consider incorporating more advanced preprocessing methods.

For further guidance, feel free to take a look at the [Image Preprocessing tutorial](https://colab.research.google.com/drive/1-ItNcRMbZBE6BCwPT-wD8m3YmHqwHxme?usp=sharing)  available in the 'Resources' section of this Kaggle competition.


In [5]:
# 3. Data Preprocessing
# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Define image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directory paths
train_dir = '/kaggle/input/bttai-ajl-2025/train/train/'

In [6]:
def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128), datagen=None):
    if datagen is None:
        datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='file_path',  
        y_col='encoded_label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        validate_filenames=False,  
        workers=1,  
        use_multiprocessing=False  
    )
    
    return generator

In [7]:
train_generator = create_generator(train_data, train_dir)
val_generator = create_generator(val_data, train_dir)

Found 2288 non-validated image filenames.
Found 572 non-validated image filenames.


## 4. Build the model


In [8]:
def build_model(input_shape=(128, 128, 3), num_classes=len(label_encoder.classes_)):
    
    inputs = keras.Input(shape=input_shape)
    
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    x = layers.Conv2D(256, (3, 3), activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2, 2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.00005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

## 5. Train the Model


In [9]:
def train_model(model, train_generator, val_generator, epochs=30):
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)]
    )
    model.save("dermatology_cnn.h5")
    return history

# Build / Training the model
model = build_model()
history = train_model(model, train_generator, val_generator)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


72/72 ━━━━━━━━━━━━━━━━━━━━ 64s 813ms/step - accuracy: 0.0791 - loss: 4.8068 - val_accuracy: 0.0192 - val_loss: 3.0633
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 57s 783ms/step - accuracy: 0.1348 - loss: 3.1717 - val_accuracy: 0.0699 - val_loss: 3.1764
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 55s 748ms/step - accuracy: 0.1529 - loss: 2.8996 - val_accuracy: 0.0699 - val_loss: 3.2666
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 54s 733ms/step - accuracy: 0.1826 - loss: 2.7714 - val_accuracy: 0.0997 - val_loss: 3.2445
Epoch 5/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 56s 767ms/step - accuracy: 0.2239 - loss: 2.6648 - val_accuracy: 0.1154 - val_loss: 3.0684
Epoch 6/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 56s 762ms/step - accuracy: 0.2545 - loss: 2.6061 - val_accuracy: 0.1626 - val_loss: 2.8590
Epoch 7/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 82s 759ms/step - accuracy: 0.2473 - loss: 2.5410 - val_accuracy: 0.2185 - val_loss: 2.7021
Epoch 8/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 56s 766ms/step - accuracy: 0.2912 - loss: 2.3664 - val_accuracy: 0.246

## 6. Make Predictions on Test Data

In [10]:
print(test_df.head())  

                                md5hash  fitzpatrick_scale  \
0  0844ae634f0e6e7ef1f73c2aeecbae0e.jpg                  2   
1  3b290d262098f761d719aa07cf36c040.jpg                  4   
2  cf561d08ac46d0fda678bff6621005ee.jpg                  2   
3  e6371069be05c6b0a95b4b3f1bacc9a5.jpg                  4   
4  f76cddb37265f97508f159078dcc7e7c.jpg                  5   

   fitzpatrick_centaur   qc  ddi_scale  
0                    2  NaN         12  
1                    3  NaN         34  
2                    3  NaN         12  
3                    3  NaN         34  
4                    5  NaN         56  


In [11]:
def preprocess_test_data(test_df, directory, target_size=(128, 128), batch_size=32):
    
    test_df = test_df.copy()  
    test_df['file_path'] = test_df['md5hash'] + '.jpg'

    test_datagen = ImageDataGenerator(rescale=1./255)

    return test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=directory,
        x_col='file_path',
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = '/kaggle/input/bttai-ajl-2025/test/test/'
test_csv = "/kaggle/input/bttai-ajl-2025/test.csv"
test_generator = preprocess_test_data(test_df, test_dir)

Found 0 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1227 invalid image filename(s) in x_col="file_path". These filename(s) will be ignored.
  warnings.warn(


## 7. Generate Predictions

In [13]:
def generate_predictions(model, test_images, file_paths, filename="submission.csv"):
    
    predictions = model.predict(test_images)
    predicted_classes = predictions.argmax(axis=1)

    label_encoder = LabelEncoder()
    label_encoder.fit(train_df["label"]) 

    predicted_labels = label_encoder.inverse_transform(predicted_classes)

    file_names = [os.path.basename(f) for f in test_images.filenames]  
    file_names = [f.replace(".jpg", "") for f in file_names] 
    
    final_df = pd.DataFrame({
        "md5hash": file_names,
        "label": predicted_labels
    })

    final_df.to_csv(filename, index=False)

In [14]:
model = keras.models.load_model("dermatology_cnn.h5")
test_df = pd.read_csv(test_csv)
test_generator = preprocess_test_data(test_df, test_dir)
generate_predictions(model, test_generator, test_df)

Found 1227 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 13s 333ms/step
